#### Imports

In [1]:
# Visualization
import matplotlib.pyplot as plt

# Builtin
from pathlib import Path
import sys
import subprocess
import os
import pdb

# Add repo path to system path for additional imports. 
# Note the relative depth from this notebook to system repo
# in case you want to move this file

nb_path = str(Path(os.path.abspath('')).parent.parent)
sys.path.insert(0, nb_path)
print(f'Added {nb_path} to system path')

# This computer git repos
from project.project_manager_module import ProjectManager

Added C:\Users\Simo\Laskenta\Git_Repos\SystemTools to system path


#### Build paths
<font color=red>**Set the root_path to your SystemTools/scripts**</font>

In [2]:
#root_path = r"/opt2/Git_Repos/SystemTools/scripts/"
root_path = r"C:\Users\Simo\Laskenta\Git_Repos\SystemTools\scripts/"

In [3]:
project = "FCN22"  
experiment = "Figure1B"  
experiment_path = Path.joinpath(Path(root_path), Path(project), experiment)

#### Call project manager
Code complexity is hidden behind the ProjectManager

In [4]:
PM = ProjectManager(
    path=experiment_path,
    input_folder="../in",
    output_folder="Comrad_sinewave",
    matlab_workspace_file="matlab_workspace_SingleSpike.mat",
    conn_skeleton_file_in="",
    conn_file_out="",
    input_filename="input_quadratic_three_units_2s_SingleSpike.mat",
    NG_name="NG3_L4_SS_L4",
    t_idx_start=2000,
    t_idx_end=-2000,
    project=project,
    experiment=experiment,
    startpoint_csv_folder="startpoint_csvs/LIF",
)

#### Copy Anat and Phys csv files from start point folder to path and update paths into the csv:s

In [5]:
anat_file_fullpath_out , phys_file_fullpath_out = PM.prepare_csvs_for_simulation()

#### Create necessary current injection files

In [6]:
PM.ct.create_current_injection(randomize=False)

#### Run CxSystem2 for single file

In [7]:
command_and_args = ["cxsystem2", "-a", str(anat_file_fullpath_out), "-p", str(phys_file_fullpath_out)]
subprocess.run(command_and_args, text=True, encoding="utf-8")

FileNotFoundError: [WinError 2] The system cannot find the file specified

### Analyze and visualize the results for single files

#### Show input and data. 
If  file_to_display = None, the function selects the most recent data file in output_folder.


In [ ]:
# file_to_display = r"[your/full/path/to/datafile].gz"  
file_to_display = None # Most recent

PM.viz.show_readout_on_input(
    results_filename=file_to_display, 
    normalize=False, 
    unit_idx_list=[0], 
    savefigname="" # Give a name, such as 'Fig1B.eps' to save the figure
)